In [1]:
from mean_average_precision import MetricBuilder
import os
import numpy as np
import glob



In [10]:
# Đường dẫn đến ground truth và prediction
gt_dir = "datasets/v2_malaria_PA7_5_class/test/labels/"
pred_dir = "runs/detect/val/labels/"

img_width = 640
img_height = 640

# Danh sách file
label_files = sorted(glob.glob(gt_dir + "*.txt"))
pred_files = sorted(glob.glob(pred_dir + "*.txt"))

print(f"Number of ground truth files: {len(label_files)}")

# Metric (dùng cho object detection)
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=True, num_classes=5)

print(MetricBuilder.get_metrics_list())

for label_file, pred_file in zip(label_files, pred_files):
    # Lấy tên ảnh (không đuôi)
    image_id = os.path.basename(label_file).replace(".txt", "")

    # Đọc ground truth
    gts = []
    with open(label_file, "r") as f:
        for line in f:
            cls, x, y, w, h = map(float, line.strip().split())
            xmin = (x - w / 2) * img_width
            xmax = (x + w / 2) * img_width
            ymin = (y - h / 2) * img_height
            ymax = (y + h / 2) * img_height
            gts.append([xmin, ymin, xmax, ymax, int(cls), 0, 0])  # 0=difficult, 0=crowd

    # Đọc prediction
    preds = []
    with open(pred_file, "r") as f:
        for line in f:
            cls, x, y, w, h, conf = map(float, line.strip().split())
            xmin = (x - w / 2) * img_width
            xmax = (x + w / 2) * img_width
            ymin = (y - h / 2) * img_height
            ymax = (y + h / 2) * img_height
            preds.append([xmin, ymin, xmax, ymax, int(cls), conf])
    gts = np.array(gts)
    preds = np.array(preds)
    # Update metric
    # if preds and gts:
    metric_fn.add(preds, gts)

# Tính mAP
res = metric_fn.value(iou_thresholds=np.arange(0.5, 1.0, 0.05), 
                recall_thresholds=np.arange(0., 1.01, 0.01), 
                mpolicy='greedy')
res.keys()


Number of ground truth files: 75
['map_2d']


dict_keys([0.5, 0.55, 0.6000000000000001, 0.6500000000000001, 0.7000000000000002, 0.7500000000000002, 0.8000000000000003, 0.8500000000000003, 0.9000000000000004, 0.9500000000000004, 'mAP'])

In [13]:
res['mAP']

0.7239159

In [8]:
from globox import AnnotationSet, BoxFormat, COCOEvaluator

IMG_DIR = "datasets/v2_malaria_PA7_5_class/test/images/"
GT_DIR = "datasets/v2_malaria_PA7_5_class/test/labels/"
PRED_DIR = "runs/detect/val3/labels/"

annotations_gt = AnnotationSet.from_yolo_v5(
    folder=GT_DIR,              # labels
    image_folder=IMG_DIR        # images
)

annotations_pred = AnnotationSet.from_yolo_v5(
    folder=PRED_DIR,            # prediction labels with confidence
    image_folder=IMG_DIR
)

evaluator = COCOEvaluator(
    ground_truths=annotations_gt, 
    predictions=annotations_pred
)

evaluations = [
    evaluator.evaluate(
        iou_threshold=t, max_detections=300, size_range=evaluator.ALL_RANGE
    )
    for t in evaluator.AP_THRESHOLDS
]

evaluations

[{'1': <globox.evaluation.EvaluationItem at 0x26f49c4d440>,
  '2': <globox.evaluation.EvaluationItem at 0x26f4a05fab0>,
  '4': <globox.evaluation.EvaluationItem at 0x26f49deaf20>,
  '3': <globox.evaluation.EvaluationItem at 0x26f49deade0>,
  '0': <globox.evaluation.EvaluationItem at 0x26f281d4a40>},
 {'1': <globox.evaluation.EvaluationItem at 0x26f49abb8d0>,
  '2': <globox.evaluation.EvaluationItem at 0x26f49ab5800>,
  '4': <globox.evaluation.EvaluationItem at 0x26f49a19620>,
  '3': <globox.evaluation.EvaluationItem at 0x26f2818f600>,
  '0': <globox.evaluation.EvaluationItem at 0x26f49a3f830>},
 {'1': <globox.evaluation.EvaluationItem at 0x26f49a71350>,
  '2': <globox.evaluation.EvaluationItem at 0x26f49a71b70>,
  '4': <globox.evaluation.EvaluationItem at 0x26f49a71bc0>,
  '3': <globox.evaluation.EvaluationItem at 0x26f49a71c10>,
  '0': <globox.evaluation.EvaluationItem at 0x26f49a71c60>},
 {'1': <globox.evaluation.EvaluationItem at 0x26f49a71d00>,
  '2': <globox.evaluation.EvaluationI

In [9]:
mAP = 0
for key, value in evaluations[0].items():
    mAP += value.ap
    print(f"{key}: {value.ap:.4f}")
mAP /= len(evaluations[0])
mAP

1: 0.4368
2: 0.8376
4: 0.9872
3: 0.9208
0: 0.7051


0.7774959651303993

                                           COCO Evaluation                                            
┏━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┓
┃ Label ┃ AP 50:95 ┃ AP 50 ┃ AP 75 ┃ AP S ┃ AP M ┃ AP L ┃ AR 1 ┃ AR 10 ┃ AR 100 ┃ AR S ┃ AR M ┃ AR L ┃
┡━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━┩
├───────┼──────────┼───────┼───────┼──────┼──────┼──────┼──────┼───────┼────────┼──────┼──────┼──────┤
│ Total │     nan% │  nan% │  nan% │ nan% │ nan% │ nan% │ nan% │  nan% │   nan% │ nan% │ nan% │ nan% │
└───────┴──────────┴───────┴───────┴──────┴──────┴──────┴──────┴───────┴────────┴──────┴──────┴──────┘